In [19]:
import tide_constituents as tc
import pandas as pd
from calendar import monthrange
import utils
from metpy.units import units

In [14]:
year = '2018'
wl_gage = (-88.2, 30.4)
q_gage = (-88.01, 31.0)

In [3]:
start = year + '0101'
end = year + '1231'
lon, lat = wl_gage
data = tc.get_water_levels(start, end, lon, lat)

In [6]:
wl = data.water_level.copy()
grouped = wl.groupby(pd.Grouper(freq='M'))

def f(group):
        return pd.DataFrame({'original': group, 'demeaned': group - group.mean()})

wl_demeaned = grouped.apply(f)
min_month = wl_demeaned.rolling(30).min().groupby(pd.Grouper(freq='M')).last()
max_month = wl_demeaned.rolling(30).max().groupby(pd.Grouper(freq='M')).last()
monthly_minmax = min_month.copy()
monthly_minmax['high'] = max_month['demeaned']
monthly_minmax = monthly_minmax[['demeaned', 'high']]
monthly_minmax.columns = ['low', 'high']
monthly_minmax['range'] = monthly_minmax.high - monthly_minmax.low
ranked = monthly_minmax.sort_values('range')

In [11]:
low, high = ranked.index[0], ranked.index[1]
for date, key in zip([low, high], ['low', 'high']):
    days = monthrange(date.year, date.month)[1]
    end = date
    start = end - pd.DateOffset(days - 1)

    prediction = tc.get_tides(start.strftime('%Y%m%d'), end.strftime('%Y%m%d'), lon, lat)
    prediction['sec'] = prediction.index
    prediction['sec'] = prediction.sec.apply(lambda x: (x - prediction.index[0]).total_seconds()).astype('int')
    prediction = prediction[['sec', 'predicted_wl']]
    utils.write_wl_bc(prediction, key)

TypeError: write_wl_bc() missing 1 required positional argument: 'out_dir'

In [24]:
lon, lat = q_gage
start, end = year + '0101', year + '1231'

q = utils.get_discharge('gage_data', start, end, (-88.2, 30.4))
q_range = [q.val.min(), q.val.mean(), q.val.max()]
q_range = [(i * units('ft^3/s')).to_base_units().magnitude for i in q_range]
#utils.write_q_bc(prediction.index[0], prediction.sec[0], prediction.sec[-1], q_range)

IndexError: list index out of range

In [23]:
q_range

[0.3284754204671999, 1.1291827456070136, 8.126934971903998]